> ⚠️ **STOP — Do Not Re-run This Notebook Cell** ⚠️  
> The cleaning script in `02-image-filtering-demo.ipynb` has already executed successfully.  
> Re-running will only duplicate work and waste compute time.

# 🧹 Cleaning Report (SCIN Dataset)

| Check                         | Result |
|-------------------------------|--------|
| 🔍 Corrupted images           | **0** detected |
| 🔍 Blurry images              | **0** detected |
| 🔄 Duplicate images           | All flagged duplicates were removed |
| ⛔ Missing images in metadata | Original single missing entry handled |

All surviving files passed integrity and basic quality tests.

## 📂 Clean Data Locations

- **Images:**  
  `/content/drive/Shared drives/Skinterest-2b/data/images/`  🖼️
- **Cleaned metadata CSV:**  
  `/content/drive/Shared drives/Skinterest-2b/data/SCIN_CLEAN.csv`  ✅
- **Removed items (for audit):**  
  `/content/drive/Shared drives/Skinterest-2b/data/trash/`  🚮

Keep using these paths for downstream preprocessing and model training.


In [ ]:
# ─── install missing library ───
!pip install --quiet imagehash

# optional: also ensure its Pillow dependency is up-to-date
!pip install --quiet --upgrade Pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 5.3 MB/s eta 0:00:00


In [ ]:
# setup imports
# Mount Google Drive (everyone must run once per Colab session)
from google.colab import drive
drive.mount('/content/drive')

# Core libs
import os, glob, json, shutil, hashlib, warnings
import pandas as pd
from PIL import Image, UnidentifiedImageError
import cv2                # OpenCV for blur detection
import imagehash          # Perceptual hashing for dup detection
warnings.filterwarnings('ignore')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# -------- paths --------
ROOT = '/content/drive/Shared drives/Skinterest-2b/data'
IMG_DIR  = f'{ROOT}/images'          # 🖼️ already-unzipped images
CSV_IN   = f'{ROOT}/SCIN Working Merged Spreadsheet.csv'
CSV_OUT  = f'{ROOT}/SCIN_CLEAN.csv'
TRASH    = f'{ROOT}/trash'           # 🚮 store removed imgs (review later)
os.makedirs(TRASH, exist_ok=True)

# -------- quality thresholds --------
BLUR_THRESH = 100        # Laplacian variance < 100 → blurry


In [ ]:
def is_corrupted(path):
    try:
        img = Image.open(path)
        img.verify()     # raises if file is broken
        return False
    except (UnidentifiedImageError, OSError):
        return True

def is_blurry_cv(path, thresh=BLUR_THRESH):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return True
    return cv2.Laplacian(img, cv2.CV_64F).var() < thresh

def file_hash(path, algo='sha1', chunk=1<<20):
    h = hashlib.new(algo)
    with open(path,'rb') as f:
        for blk in iter(lambda: f.read(chunk), b''):
            h.update(blk)
    return h.hexdigest()


In [ ]:
hash_set = {}          # hash → path keeper
remove_list = []       # (path, reason)

for img_path in glob.glob(f'{IMG_DIR}/**/*.jpg', recursive=True):
    # 1. broken?
    if is_corrupted(img_path):
        remove_list.append((img_path, 'corrupted'))
        continue
    # 2. blurry?
    if is_blurry_cv(img_path):
        remove_list.append((img_path, 'blurry'))
        continue
    # 3. duplicate?
    h = file_hash(img_path)
    if h in hash_set:
        remove_list.append((img_path, f'duplicate_of:{os.path.basename(hash_set[h])}'))
    else:
        hash_set[h] = img_path


In [ ]:
for path, reason in remove_list:
    dest = os.path.join(TRASH, os.path.basename(path))
    shutil.move(path, dest)
print(f'Removed {len(remove_list)} images → {TRASH}')


Removed 0 images → /content/drive/Shared drives/Skinterest-2b/data/trash


In [ ]:
df = pd.read_csv(CSV_IN)
img_exists = set(os.listdir(IMG_DIR))   # filenames that survived

def row_is_valid(row):
    img_cols = [c for c in row.index if c.startswith('image_')]
    for c in img_cols:
        if pd.isna(row[c]):     # no path = OK
            continue
        if os.path.basename(row[c]) not in img_exists:
            return False        # missing / removed
    return True

clean_df = df[df.apply(row_is_valid, axis=1)].reset_index(drop=True)
clean_df.to_csv(CSV_OUT, index=False)
print(f'Saved cleaned metadata → {CSV_OUT}\nRows kept: {len(clean_df)} / {len(df)}')


Saved cleaned metadata → /content/drive/Shared drives/Skinterest-2b/data/SCIN_CLEAN.csv
Rows kept: 0 / 770


In [ ]:
import collections, pprint
stats = collections.Counter([r for _, r in remove_list])
print('Removal reasons:\n')
pprint.pprint(stats)
print(f'\nFinal image count: {len(hash_set)}')


Removal reasons:

Counter()

Final image count: 0
